In [ ]:
import pandas as pd
import numpy as np

In [ ]:
posts = pd.read_csv("posts.csv")
views = pd.read_csv("views.csv")
users = pd.read_csv("users.csv")

In [ ]:
users.head()

In [ ]:
users.describe()

In [ ]:
views.head()

In [ ]:
posts.head()

In [ ]:
posts.sort_values("_id", axis = 0, ascending = False)

In [ ]:
views.sort_values("post_id", axis = 0, ascending = False)

In [ ]:
posts = posts.rename(columns = {"_id": "post_id"})

In [ ]:
posts.head()

In [ ]:
merged_views_df = views.merge(posts, on = "post_id")

In [ ]:
merged_views_df.head()

In [ ]:
merged_views_df = merged_views_df.drop(["timestamp"], axis = 1)
merged_views_df.head()

In [ ]:
merged_views_df.describe()

In [ ]:
merged_views_df.info()

In [ ]:
merged_views_df.dropna(axis = 0, inplace = True)

In [ ]:
merged_views_df.info()

In [ ]:
merged_views_df = merged_views_df.reset_index()

In [ ]:
post_viewCount = views.groupby("post_id")
post_viewCount.head()

In [ ]:
post_viewCount = post_viewCount.count().reset_index().rename(columns = {"timestamp" : "view_count"})

In [ ]:
post_viewCount.head()

In [ ]:
post_viewCount = post_viewCount[["post_id", "view_count"]]

In [ ]:
merged_views_df.head()

In [ ]:
views_count_df = merged_views_df.merge(post_viewCount, left_on = "post_id", right_on = "post_id", how = "left" )

In [ ]:
no_of_views = [0]*views_count_df.shape[0]

In [ ]:
views_count_df["no_of_views"] = pd.Series(no_of_views, index = views_count_df.index)

In [ ]:
views_count_df.head()

In [ ]:
views_count_df["view_count"].describe()

In [ ]:
threshold_view = 5
popular_blogs = views_count_df.query("view_count >= @threshold_view")
popular_blogs = popular_blogs.sort_values("view_count", ascending = True)

In [ ]:
popular_blogs["user_id"].describe()

In [ ]:
for i in range(users.shape[0]):
    for j in range(views_count_df.shape[0]):
        if users["_id"][i] == views_count_df["user_id"][j]:
            views_count_df["no_of_views"][j] += 1   

In [ ]:
views_count_df.head()

In [ ]:
views_count_df["no_of_views"].describe()

In [ ]:
views_count_df.sort_values("no_of_views", axis = 0, ascending = True)

In [ ]:
post_views = views_count_df.pivot_table(index='title',columns='user_id', values = "no_of_views").fillna(0)
post_views

In [ ]:
post_views = post_views.sort_values("title", axis = 0, ascending = False)

In [ ]:
post_views.shape

In [ ]:
post_views.index[145]

In [ ]:
from scipy.sparse import csr_matrix

pivot_matrix = csr_matrix(post_views.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(pivot_matrix)

In [ ]:
def give_index(title):
    for index in range(post_views.shape[0]):
        if title == post_views.index[index]:
            return index       

In [ ]:
title = 'trekking'
index = give_index(title)
distances, indices = model_knn.kneighbors(post_views.iloc[index,:].values.reshape(1, -1), n_neighbors = 6)
recomm_list = []
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for post- {0}:\n'.format(post_views.index[index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, post_views.index[indices.flatten()[i]], distances.flatten()[i]))
        recomm_list.append(post_views.index[indices.flatten()[i]])

In [ ]:
recomm_list